In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import entropy


from qiskit import Aer, execute, QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.providers.aer.noise import NoiseModel
from qiskit.test.mock import FakeVigo

### Parameters of simulation
We define the target probability distribution, the number of measurements and the noise model used in our simulations.

In [2]:
# Global parameters
BACKEND = Aer.get_backend('qasm_simulator')

TARGET = {'01': .5, 
          '10': .5}

# Noise model
device = FakeVigo()
NOISE_MODEL = NoiseModel.from_backend(device)
BASIS_GATES = NOISE_MODEL.basis_gates

### Ansatz
Our variatonal form consist of 'RY' and 'CNOT' gates. It's 'inspired' from the fact that

$$ CNOT_{0,1} \cdot \big(X_1 \otimes H_0\big)  |00 \rangle = \frac{|01 \rangle + |10 \rangle}{2} $$

and that $ R_y(\pi / 2) |0 \rangle = H |0 \rangle, \quad R_y(\pi) |0 \rangle = X |0 \rangle $

In [3]:
# define variatonal circuit
def ansatz():
    phi = ParameterVector(name='φ', 
                          length=2)

    qc = QuantumCircuit(2)
    qc.ry(phi[0], 0)
    qc.ry(phi[1], 1)
    qc.cx(0, 1)
    
    return qc, list(qc.parameters)


qc, params = ansatz()
print('Variatonal form')
print(qc)

Variatonal form
     ┌──────────┐     
q_0: ┤ RY(φ[0]) ├──■──
     ├──────────┤┌─┴─┐
q_1: ┤ RY(φ[1]) ├┤ X ├
     └──────────┘└───┘


### Cost function
We try experiments with $L_1$-distance and  Kullback-Leibler divergence of the output and target probability distributions as loss functions. 

We add the expectation value of the $ X \otimes X $ operator as a regularizer term to ensure that the output state is $ |\psi_+ \rangle = \frac{|01 \rangle + |10 \rangle}{2} $ and not $ |\psi_- \rangle = \frac{|01 \rangle - |10 \rangle}{2} $,   since $$ \langle \psi_+ | X \otimes X |\psi_+ \rangle = 1 $$ but $ \quad \langle \psi_- | X \otimes X |\psi_- \rangle = -1 $.

This means our total cost function is, for example:

$$ L(\theta) = ||p_{\theta} - q||_{L_1} + \lambda \cdot \big(1 - \langle \psi(\theta) | X \otimes X |\psi(\theta) \rangle\big) $$

where $p_{\theta},  q$ are probability distributions on $ \{00, 01, 10, 11\}$ defined by
$$ p_{\theta}(b_0 b_1) = \big| \langle \psi(\theta) | b_0 b_1 \rangle \big|^2 \quad \text{ and } \quad q(01) = q(10) = \frac{1}{2} $$

Of course, we only have access to estimates of the probabilities and expectation values through our measurements.

As a side note, to calculate the expectation value of $ X \otimes X $ operator we observe that

$$ \langle \psi(\theta) | X \otimes X |\psi(\theta) \rangle = \langle \psi(\theta) | H^{\otimes 2} \big(Z \otimes Z \big) H^{\otimes 2} |\psi(\theta) \rangle $$

and that's why we append $ H^{\otimes 2} $ in the circuit and measure expectation of $ Z \otimes Z $ through measurements in computational basis.

In [4]:
# KL - Divergence of 2 dictionaries
def KL(count, target):
    p = np.zeros(4)
    q = np.zeros(4)
    
    for i, key in enumerate(['00', '01', '10', '11']):
        p[i] = count.get(key, 0.0)
        q[i] = target.get(key, 0.0)
        
    return entropy(q, p)


# define cost function
def cost(phi, noise=False, L1=True, n_meas=1024, verbose=False):
    
    # circuit 
    circ = qc.bind_parameters(dict(zip(params, phi)))
    circ.measure_all()
    
    # simulate 
    if noise:
        result = execute(circ, backend=BACKEND, shots=n_meas,
                         noise_model=NOISE_MODEL, basis_gates=BASIS_GATES).result()
    else:
        result = execute(circ, backend=BACKEND, shots=n_meas).result()
        
    counts = result.get_counts()
    
    # L1 - norm
    if L1:
        cost = 0.0
        for key in ['00', '01', '10', '11']:
            cost += np.abs(counts.get(key, 0.0) / n_meas - TARGET.get(key, 0.0))
            
    # KL - Divergence
    else:
        cost = KL(counts, TARGET)
    
    # show counts during optimization
    if verbose:
        print(counts)
        
    return cost


# 'XX' expectation as regularizer term
def XXRegularizer(phi, noise=False, n_meas=1024):
    
    # circuit 
    circ = qc.bind_parameters(dict(zip(params, phi)))
    circ.h([0, 1])
    circ.measure_all()
    
    # simulate 
    if noise:
        result = execute(circ, backend=BACKEND, shots=n_meas,
                         noise_model=NOISE_MODEL, basis_gates=BASIS_GATES).result()
    else:
        result = execute(circ, backend=BACKEND, shots=n_meas).result()
        
    counts = result.get_counts()
    
    
    expectation = 0.0
    for key in ['00', '11']:
        expectation += counts.get(key, 0.0) / n_meas
        
    for key in ['01', '10']:
        expectation -= counts.get(key, 0.0) / n_meas
        
    return (1 - expectation)

### Optimize
We choose COBYLA as our optimization method since gradient-based methods do not work well due to the noise present in our simulations and statistical noise coming through the limited number of measurements. 

In [12]:
# random initial choice
phi = np.random.uniform(0, 2 * np.pi, len(params))

SHOTS = 1024
# hyperparameter
lam = .7
cost_fn = lambda phi: cost(phi, 
                           noise=True, verbose=True, n_meas=SHOTS) \
                     + lam * XXRegularizer(phi, 
                                           noise=True, n_meas=SHOTS)
res = minimize(cost_fn, phi, 
               method='COBYLA')
res

{'10': 292, '11': 125, '01': 98, '00': 509}
{'10': 665, '11': 35, '01': 162, '00': 162}
{'10': 821, '11': 3, '01': 1, '00': 199}
{'10': 939, '11': 2, '01': 50, '00': 33}
{'10': 583, '11': 40, '01': 270, '00': 131}
{'10': 207, '11': 102, '01': 670, '00': 45}
{'10': 286, '11': 165, '01': 108, '00': 465}
{'10': 736, '11': 8, '01': 234, '00': 46}
{'10': 552, '11': 66, '01': 198, '00': 208}
{'10': 470, '11': 62, '01': 378, '00': 114}
{'10': 392, '11': 48, '01': 511, '00': 73}
{'10': 305, '11': 96, '01': 546, '00': 77}
{'10': 481, '11': 21, '01': 481, '00': 41}
{'10': 477, '11': 8, '01': 509, '00': 30}
{'10': 526, '11': 4, '01': 473, '00': 21}
{'10': 413, '11': 8, '01': 572, '00': 31}
{'10': 463, '11': 9, '01': 530, '00': 22}
{'10': 470, '11': 11, '01': 522, '00': 21}
{'10': 453, '11': 7, '01': 539, '00': 25}
{'10': 437, '11': 14, '01': 554, '00': 19}
{'10': 440, '11': 9, '01': 545, '00': 30}
{'10': 445, '11': 7, '01': 547, '00': 25}
{'10': 443, '11': 15, '01': 536, '00': 30}
{'10': 436, '11

     fun: 0.1474609375
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 37
  status: 1
 success: True
       x: array([3.27684627, 7.92084137])

### Simulate circuit with optimized parameters
After we find the optimal values of parameters, we compare the output state of our circuit with the target $ |\psi_+ \rangle$ state.

In [16]:
from qiskit.quantum_info import state_fidelity

# Target state
target_state = (1 / np.sqrt(2)) * np.array([0, 1, 1, 0])

backend = Aer.get_backend('statevector_simulator')

def fidelity(phi):
    # output state
    circ = qc.bind_parameters(dict(zip(params, phi)))

    result = execute(circ, backend).result()
    output_state = result.get_statevector()
    
    return output_state, state_fidelity(output_state, target_state)



output_state, F = fidelity(res.x)

print('Output state:\n ', np.around(output_state, decimals=5))
print('Target state:\n ', np.around(target_state, decimals=5))
print('\nFidelity:\n ', F)

Output state:
  [ 0.07752+0.j -0.70888+0.j -0.6966 +0.j  0.07888+0.j]
Target state:
  [0.      0.70711 0.70711 0.     ]

Fidelity:
  0.9876931745159916


### Benchmark with different number of measurements
By increasing the number of measurements, we are able to achieve smaller error between output and target probability distributions and improve the fidelity of our state with $ |\psi_+ \rangle $. This is expected since with few measurements our statistical estimates are not accurate enough.

In [24]:
lam = 2

# random initial choice
phi_init = np.random.uniform(0, 2 * np.pi, len(params))

for shots in [1, 10, 100, 1000]:

    cost_fn = lambda phi: cost(phi, 
                               noise=True, n_meas=shots) \
                         + lam * XXRegularizer(phi, 
                                               noise=True, n_meas=shots)
    res = minimize(cost_fn, phi_init, 
                   method='COBYLA')
    
    _, F = fidelity(res.x)
    
    print('\nShots: ', shots)
    print('       loss: {:.3}'.format(res.fun))
    print('   fidelity: {:.3}'.format(F))


Shots:  1
       loss: 1.0
   fidelity: 0.817

Shots:  10
       loss: 0.4
   fidelity: 0.955

Shots:  100
       loss: 0.5
   fidelity: 0.876

Shots:  1000
       loss: 0.262
   fidelity: 0.993


## State preparation with VQE in Qiskit-Aqua
In this section, we provide an alternative solution using Qiskit Aqua.

We rephrase the problem as finding the lowest energy eigenstate of a rank-1 Hamiltonian and we use VQE algorithm built in Qiskit.
$$ H = - |\psi_+ \rangle \langle \psi_+| , \qquad |\psi_+ \rangle = \frac{|01 \rangle + |10 \rangle}{\sqrt{2}}$$

If we end up in the ground state of $H$, then our circuit will return $ |01 \rangle, |10 \rangle $ with equal probability.

In [49]:
from qiskit.aqua.operators import MatrixOp, VectorStateFn
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua import QuantumInstance


H = - VectorStateFn(target_state).to_density_matrix()
Hp = MatrixOp(H).to_pauli_op()

optimizer = SPSA(max_trials=100)

var_form, _ = ansatz()

vqe = VQE(Hp, var_form, optimizer)
vqe_result = vqe.run(QuantumInstance(backend=BACKEND, 
                                     noise_model=NOISE_MODEL))

vqe_result

{'optimal_parameters': {Parameter(φ[0]): 1.5671007528561058, Parameter(φ[1]): -3.089639850187522}, 'optimal_point': array([ 1.56710075, -3.08963985]), 'optimal_value': -0.94091796875, 'optimizer_time': 30.742521047592163, 'eigenvalue': (-0.94091796875+0j), 'eigenstate': {'11': 4, '00': 32, '10': 492, '01': 496}, 'cost_function_evals': 241}

In [53]:
circ = var_form.bind_parameters(vqe_result.optimal_parameters)

result = execute(circ, backend).result()
output_state = result.get_statevector()

print('Fidelity: {:.3}'.format(state_fidelity(output_state, target_state)))

Fidelity: 0.999
